In [1]:

import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
import pandas as pd
import numpy as np
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
from pyspark.sql import SparkSession
spark= SparkSession.builder. \
appName("Major Project").\
master("local[1]").\
config("spark.memory.offHeap.enabled","true"). \
config("spark.memory.offHeap.size","10g").getOrCreate()
spark




# Regression 

In [3]:
df=spark.read.csv(r"C:/Users/abhi/Desktop/project feb2023/major project code/small dataset/train2.csv",inferSchema=True,header=True)

print('Number of rows: \t', df.count())
print('Number of columns: \t', len(df.columns))

#df.show(10)
df.columns

In [4]:
#dataype of input data 
df.printSchema()

root
 |-- bidirectional_duration_ms: integer (nullable = true)
 |-- bidirectional_packets: integer (nullable = true)
 |-- bidirectional_bytes: integer (nullable = true)
 |-- src2dst_first_seen_ms: long (nullable = true)
 |-- src2dst_last_seen_ms: long (nullable = true)
 |-- src2dst_packets: integer (nullable = true)
 |-- src2dst_bytes: integer (nullable = true)
 |-- dst2src_first_seen_ms: long (nullable = true)
 |-- dst2src_last_seen_ms: long (nullable = true)
 |-- dst2src_packets: integer (nullable = true)
 |-- dst2src_bytes: integer (nullable = true)
 |-- dst2src_duration_ms: integer (nullable = true)
 |-- src2dst_duration_ms: integer (nullable = true)
 |-- Target: integer (nullable = true)



In [5]:
df.groupBy('Target').count().show()

+------+-------+
|Target|  count|
+------+-------+
|     1|6637795|
|     0|    555|
+------+-------+



In [6]:
from pyspark.sql.types import *
for col in df.columns:
    df = df.withColumn(col, df[col].cast(IntegerType()))

In [7]:
import pyspark.sql.functions as f
from pyspark.sql.types import *

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import OneHotEncoder, StandardScaler, VectorAssembler, StringIndexer, Imputer

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:

pipe_stages= []

# create the un-standardized features vector
featureCols= ['bidirectional_duration_ms','bidirectional_packets','bidirectional_bytes',
              'src2dst_packets','src2dst_bytes','dst2src_packets','dst2src_bytes',
              'dst2src_duration_ms','src2dst_duration_ms']
assembler= VectorAssembler(inputCols= featureCols,
                           outputCol= "features",
                           handleInvalid="keep")
pipe_stages += [assembler]

# scale all features. 
ss= StandardScaler(inputCol="features",
                   outputCol="features_scale",
                   withMean= False,
                   withStd=True)
pipe_stages += [ss]

rf = RandomForestClassifier(labelCol="Target", featuresCol="features_scale", seed = 8464,
                            numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)
pipe_stages += [rf]

pipe= Pipeline(stages= pipe_stages)

In [ ]:
mod = pipe.fit(df)

In [ ]:
df2 = mod.transform(df)

In [ ]:
mod.stages[-1].featureImportances
#df= pipe.fit(df).transform(df)

In [ ]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))


In [ ]:
 ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features").head(10)

In [ ]:
varlist = ExtractFeatureImp(mod.stages[-1].featureImportances, df2, "features")

In [ ]:
varidx = [x for x in varlist['idx'][0:4]]

In [ ]:
varidx  

In [ ]:
pipe_stages= []

# create the un-standardized features vector
featureCols= ['bidirectional_duration_ms','bidirectional_packets','bidirectional_bytes',
              'src2dst_packets','src2dst_bytes','dst2src_packets','dst2src_bytes',
              'dst2src_duration_ms','src2dst_duration_ms']
assembler= VectorAssembler(inputCols= featureCols,
                           outputCol= "features",
                           handleInvalid="keep")
pipe_stages += [assembler]

# scale all features. 
ss= StandardScaler(inputCol="features",
                   outputCol="features_scale",
                   withMean= False,
                   withStd=True)
pipe_stages += [ss]

rf = RandomForestClassifier(labelCol="Target", featuresCol="features_scale", seed = 8464,
                            numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)
pipe_stages += [rf]

pipe= Pipeline(stages= pipe_stages)

In [ ]:
 reference
    
    
    
#https://github.com/timlrx/timlrx.com/blob/master/data/blog/2018-06-19-
#feature-selection-using-feature-importance-score-creating-a-pyspark-estimator.md


#https://github.com/rampanyam/Predict-Next-Day-Rain-in-Australia-Classification
#-Problem/blob/main/Predicting_Rain_Tomorrow_Kaggle_Raghuram.ipynb


#https://haya-toumy.gitbook.io/spark-notes/pyspark/pyspark/random-forest-classifier-example
#Creating a Pipeline to Do All Feature Engineering